In [25]:
import numpy as np
import os
import shutil
import tensorflow as tf
def load_im(path):
    cache_subdir = os.path.join(os.getcwd(), "caches/")
    img = tf.keras.preprocessing.image.load_img(tf.keras.utils.get_file(origin=img_path, cache_subdir=cache_subdir), target_size=(200, 200), color_mode = "grayscale")
    # Convert the image to a numpy array
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    # Normalize the pixel values
    x = x / 255.0
    shutil.rmtree(cache_subdir)
    return x
    
# img_path = "/mnt/e/Document/HK1-2023-2024/TLCN/BuildModel/Data/val/Viral Pneumonia/09.jpeg"
# img_path = "https://firebasestorage.googleapis.com/v0/b/practicefirebase-f0570.appspot.com/o/images%2FPituitary%2FTe-pi_0052.jpg?alt=media&token=cad5d596-f37a-4e13-b855-932437d401d7"
img_path = "https://firebasestorage.googleapis.com/v0/b/practicefirebase-f0570.appspot.com/o/images%2FTuberculosis%2Ftest_0_3036.jpeg?alt=media&token=5f8b7d56-c19e-4b1c-8ed7-3f51054296fc"
# img_path = "E:/Document/HK1-2023-2024/TLCN/BuildModel/Data/val/Corona Virus Disease/test_0_9575.jpeg"
# img = load_img(img_path, target_size=(200, 200), color_mode = "grayscale")

model = tf.keras.models.load_model('model_weights.h5')
# Make the prediction
prediction = model.predict(load_im(img_path))

# Get the class label with the highest probability
predicted_class = np.argmax(prediction)

print(predicted_class)

2023-11-06 19:23:47.845202: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.04GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


1/1 [==============================] - 2s 2s/step
3


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from tkinter import messagebox
from PIL import ImageTk, Image
import numpy as np
import csv
import threading

# Define the class names
class_names = []
with open('names.csv', newline='') as f:
    reader = csv.DictReader(f)
    for row in reader:
        class_names.append(row['name'])

# variable
file_path = None
origin_image = None
# Load the saved model
model = tf.keras.models.load_model('model_weights.h5')
img_height = img_width = 200

# Create a function to preprocess the image
def preprocess_image(image):
    image_array = img_to_array(image)
    image_array = np.expand_dims(image_array, axis=0)
    image_array = image_array / 255.0
    return image_array

# Create a function to classify the image
def classify_image():
    global origin_image
    global class_names
    if origin_image == None:
        # Show an error message if there was a problem loading the image
        messagebox.showerror('Error', 'Please upload image!')
        return
    # Load the image
    # image = origin_image.copy()
    # Preprocess the image
    image_array = preprocess_image(origin_image)
    # Use the model to make predictions
    predictions = model.predict(image_array)
    canvasInfor.delete('all')
    text = canvasInfor.create_text(90, 50, anchor=tk.CENTER, text = "Predict information \n", font=("Arial", 14))
    index = 0
    for name in class_names:
        text = canvasInfor.create_text(10, 90 + index * 25, anchor=tk.W, text = "{0}: {1}% \n ".format( name, round( predictions[0][index]*100, 2)), font=("Arial", 14))
        index+=1
    origin_image = None

# Create a function to handle the upload button click
def handle_upload():
    # Open a file dialog to select the image file
    global file_path
    global origin_image
    file_path = filedialog.askopenfilename()
    # Check if a file was selected
    if file_path:
        try:
            # Load the image and display it in the GUI
            # origin_image = Image.open(file_path)
            origin_image = load_img(file_path, target_size=(200, 200), color_mode = "grayscale")
            image = origin_image.copy()
            image = image.resize((400, 400))
            image_tk = ImageTk.PhotoImage(image)
            canvas.itemconfig(canvas_image, image=image_tk)
            canvas.image = image_tk
        except:
            # Show an error message if there was a problem loading the image
            messagebox.showerror('Error', 'Could not open the image file.')

    
# Create the Tkinter window
window = tk.Tk()
window.title('Disease Classifier')
window_width = 900
window_height = 650
screen_width = window.winfo_screenwidth()
screen_height = window.winfo_screenheight()
x = (screen_width - window_width) // 2
y = (screen_height - window_height) // 2
window.geometry(f"{window_width}x{window_height}+{x}+{y}")


# Create a canvas to display the image
canvas = tk.Canvas(window, width=400, height=400, bd=2, relief='solid')
canvas.place(x=50, y = 100 + 100)
canvas_image = canvas.create_image(0, 0, anchor=tk.NW)

# Create a canvas to display infor
canvasInfor = tk.Canvas(window, width=250, height=400, bd=1, relief='solid')
canvasInfor.place(x=500, y = 100 + 100)

# Create a button to upload the image
upload_button = tk.Button(window, text='Upload Image', command=handle_upload, width=12)
upload_button.place(x=760, y = 100 + 100)

classify_button = tk.Button(window, text='Classify', command=classify_image, width=12)
classify_button.place(x=760, y = 200 + 100)

# Start the GUI main loop
window.mainloop()